In [1]:
import math
import tensorflow as tf
import numpy as np
import pylab as plt

c:\users\ligua\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\ligua\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\ligua\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\ligua\appd

In [2]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# scale data
def scale(X, X_min, X_max):
    return (X - X_min)/(X_max-X_min)

NUM_FEATURES = 21
NUM_CLASSES = 3

learning_rate = 0.01
epochs = 5000
num_neurons = 10
seed = 10
np.random.seed(seed)

In [3]:
# Define the feedforward network
def ffn(input_feature, hidden_layer_neuron_num):
    with tf.name_scope('hidden_layer'):
        weights = tf.Variable(
            tf.truncated_normal(
                [NUM_FEATURES, hidden_layer_neuron_num], 
                stddev=1.0/math.sqrt(float(NUM_FEATURES))),
            name='weights')
        biases = tf.Variable(
            tf.zeros([hidden_layer_neuron_num]),
            name='biases')
        hidden = tf.nn.relu(tf.matmul(input_feature, weights) + biases)
    with tf.name_scope('softmax_output'):
            weights = tf.Variable(
                tf.truncated_normal(
                    [hidden_layer_neuron_num, NUM_CLASSES], 
                    stddev=1.0/math.sqrt(float(NUM_FEATURES))),
                name='weights')
            biases = tf.Variable(
                tf.zeros([NUM_CLASSES]),
                name='biases')
            logits = tf.matmul(hidden, weights) + biases
    return logits

def train(batch_size):
    train_input = np.genfromtxt('ctg_data_cleaned.csv', delimiter= ',')
    trainX, train_Y = train_input[1:, :21], train_input[1:,-1].astype(int)
    trainX = scale(trainX, np.min(trainX, axis=0), np.max(trainX, axis=0))

    trainY = np.zeros((train_Y.shape[0], NUM_CLASSES))
    trainY[np.arange(train_Y.shape[0]), train_Y-1] = 1 #one hot matrix


    # experiment with small datasets
    trainX = trainX[:1000]
    trainY = trainY[:1000]
    

In [4]:
    trainX

    

NameError: name 'trainX' is not defined

In [ ]:
    n = trainX.shape[0]
    x = tf.placeholder(tf.float32, [None, NUM_FEATURES])
    y_ = tf.placeholder(tf.float32, [None, NUM_CLASSES])
    y = ffn(x, 10)

    with tf.name_scope('cross_entropy'):
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_, logits=y)
        cross_entropy = tf.reduce_mean(cross_entropy)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        global_step = tf.Variable(0, name='global_step', trainable=False)

    with tf.name_scope('accuracy'):
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
        correct_prediction = tf.cast(correct_prediction, tf.float32)
        accuracy = tf.reduce_mean(correct_prediction)

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        train_acc = []
        
        for i in range(1, epochs + 1):
            first_index = (i - 1) * batch_size
            last_index = i * batch_size
            if last_index > len(trainX):
                last_index = len(trainX)
            
            train_op.run(feed_dict={x: trainX[first_index:last_index], y_: trainY[first_index:last_index]})
            train_acc.append(accuracy.eval(feed_dict={x: trainX, y_: trainY}))

            if i % 100 == 0:
                print('iter %d: accuracy %g'%(i, train_acc[i]))

    plt.figure(1)
    plt.plot(range(epochs), train_acc)
    plt.xlabel(str(epochs) + ' iterations')
    plt.ylabel('Train accuracy')
    plt

In [ ]:
    def main():
        batch_size = 32
        train(batch_size)